In [ ]:
#Procedure for allocating the population aged 65 and above to buildings

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.wkt import loads, dumps  # Import the dumps function

# Load the data
bgri = gpd.read_file("BGRI2021_1312.gpkg")
edificios = pd.read_csv("edificios.csv")

# Convert 'edificios' to GeoDataFrame, ensuring the geometry column is properly parsed
edificios['geometry'] = edificios['geometry'].apply(loads)
edificios = gpd.GeoDataFrame(edificios, geometry='geometry')

# Define CRS and convert to EPSG:3763
edificios.set_crs(epsg=4326, inplace=True)
edificios = edificios.to_crs(epsg=3763)
bgri = bgri.to_crs(epsg=3763)

# Remove 'index_right' column if it exists
if 'index_right' in edificios.columns:
    edificios.drop(columns=['index_right'], inplace=True)
if 'index_right' in bgri.columns:
    bgri.drop(columns=['index_right'], inplace=True)

# Fix potential topological errors in geometries
edificios['geometry'] = edificios['geometry'].buffer(0)
bgri['geometry'] = bgri['geometry'].buffer(0)

# Perform spatial join to associate each building with a statistical subsection
edificios = gpd.sjoin(edificios, bgri[['SUBSECCAO', 'geometry']], how='left', predicate='intersects')

# Merge buildings with population data (65 or older) from BGRI
edificios = edificios.merge(
    bgri[['SUBSECCAO', 'N_INDIVIDUOS_65_OU_MAIS']],
    on='SUBSECCAO',
    how='left'
)

# Calculate area of each building and total area per subsection
edificios['area'] = edificios['geometry'].area
soma_areas = edificios.groupby('SUBSECCAO')['area'].sum().reset_index()
soma_areas.rename(columns={'area': 'TOTAL_AREA_SE'}, inplace=True)
edificios = edificios.merge(soma_areas, on='SUBSECCAO', how='left')

# --- Distribution for the 65+ age group ---
edificios['POP_65_OU_MAIS_PROPORCIONAL'] = (
    (edificios['area'] / edificios['TOTAL_AREA_SE']) * edificios['N_INDIVIDUOS_65_OU_MAIS']
)
edificios['POP_65_OU_MAIS_ARREDONDADA'] = edificios['POP_65_OU_MAIS_PROPORCIONAL'].round()

# Adjust so that the sum of rounded values per subsection matches N_INDIVIDUOS_65_OU_MAIS
ajustes_65_ou_mais = (
    edificios.groupby('SUBSECCAO')['POP_65_OU_MAIS_ARREDONDADA'].sum() -
    edificios.groupby('SUBSECCAO')['N_INDIVIDUOS_65_OU_MAIS'].first()
).reset_index()
ajustes_65_ou_mais.rename(columns={0: 'AJUSTE'}, inplace=True)

# Safely distribute the adjustment across buildings, without exceeding the total population per subsection
for _, row in ajustes_65_ou_mais.iterrows():
    subsecao = row['SUBSECCAO']
    diff = int(row['AJUSTE'])
    
    if diff == 0:
        continue

    subset = edificios[edificios['SUBSECCAO'] == subsecao]
    if subset.empty:
        continue
    
    # Sort buildings by estimated population
    subset_sorted = subset.sort_values(by='POP_65_OU_MAIS_ARREDONDADA', ascending=(diff > 0))
    
    # Ensure the sum does not exceed the total population of the subsection
    total_subseccao = subset['N_INDIVIDUOS_65_OU_MAIS'].iloc[0]
    current_total = subset_sorted['POP_65_OU_MAIS_ARREDONDADA'].sum()
    max_possible_diff = total_subseccao - current_total
    
    diff = int(min(diff, max_possible_diff))
    
    # Distribute the adjustment cyclically, ensuring no negative values
    for i in range(abs(diff)):
        idx = subset_sorted.index[i % len(subset_sorted)]
        if diff > 0:
            # Decrease population, ensuring it does not go negative
            if edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] > 0:
                edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] -= 1
        else:
            # Increase population (if there is a deficit)
            if edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] < total_subseccao:
                edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] += 1

# --- Final adjustment block per SUBSECCAO ---
# Ensure that, for each SUBSECCAO, the sum matches the official total
for sub in edificios['SUBSECCAO'].unique():
    subset = edificios[edificios['SUBSECCAO'] == sub]
    official = subset['N_INDIVIDUOS_65_OU_MAIS'].iloc[0]
    current = subset['POP_65_OU_MAIS_ARREDONDADA'].sum()
    diff_final = int(current - official)
    if diff_final > 0:
        # If there is excess, subtract it from buildings with highest values
        subset_sorted = subset.sort_values(by='POP_65_OU_MAIS_ARREDONDADA', ascending=False)
        for i in range(diff_final):
            idx = subset_sorted.index[i % len(subset_sorted)]
            if edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] > 0:
                edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] -= 1
    elif diff_final < 0:
        # If there is a deficit, increment population as needed
        subset_sorted = subset.sort_values(by='POP_65_OU_MAIS_ARREDONDADA', ascending=True)
        for i in range(abs(diff_final)):
            idx = subset_sorted.index[i % len(subset_sorted)]
            edificios.loc[idx, 'POP_65_OU_MAIS_ARREDONDADA'] += 1

# Ensure there are no negative values after adjustments
edificios['POP_65_OU_MAIS_ARREDONDADA'] = edificios['POP_65_OU_MAIS_ARREDONDADA'].clip(lower=0)

# Convert coordinates to EPSG:4326 for Folium visualization
edificios = edificios.to_crs(epsg=4326)
bgri = bgri.to_crs(epsg=4326)

# Calculate total 65+ population per SUBSECCAO for visualization
pop_bgri = (
    edificios.groupby('SUBSECCAO')['POP_65_OU_MAIS_ARREDONDADA']
    .sum()
    .reset_index()
    .rename(columns={'POP_65_OU_MAIS_ARREDONDADA': 'POP_TOTAL_65_OU_MAIS'})
)
bgri = bgri.merge(pop_bgri, on='SUBSECCAO', how='left')
bgri['POP_TOTAL_65_OU_MAIS'] = bgri['POP_TOTAL_65_OU_MAIS'].fillna(0)

# Define map center (approximate center of Porto)
centro = [41.14961, -8.61099]

# Create a Folium map centered on Porto
m = folium.Map(location=centro, zoom_start=14)

# Add building polygons to the map, color-coded by 65+ population
for _, row in edificios.iterrows():
    color = 'purple' if row['POP_65_OU_MAIS_ARREDONDADA'] > 10 else 'blue'
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=(
            f"Building {row['osm_id']}<br>"
            f"65+ Population: {row['POP_65_OU_MAIS_ARREDONDADA']}"
        )
    ).add_to(m)

# Add BGRI subsections to the map with transparency and 65+ population tooltips
folium.GeoJson(
    bgri,
    style_function=lambda x: {
        'fillColor': 'green',
        'color': 'green',
        'weight': 1,
        'fillOpacity': 0.1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['POP_TOTAL_65_OU_MAIS'], 
        aliases=['Total Population 65+']
    )
).add_to(m)

# Verify population totals
pop_total_adjusted = edificios['POP_65_OU_MAIS_ARREDONDADA'].sum()
pop_total_bagri = bgri['N_INDIVIDUOS_65_OU_MAIS'].sum()

print(f"Total adjusted 65+ population: {pop_total_adjusted}")
print(f"Total BGRI 65+ population: {pop_total_bagri}") 

# Display the map in the Notebook
display(m)

# Save the result as CSV with geometry converted to WKT
edificios['geometry_wkt'] = edificios['geometry'].apply(dumps)
edificios[['osm_id', 'SUBSECCAO', 'area', 'POP_65_OU_MAIS_ARREDONDADA', 'geometry_wkt']].to_csv(
    'pop_reconstruido_65_ou_mais.csv',
    index=False
)

print("arquivo.csv")